Cyclingrace results

Cyclingrace is an amateaur road race series.
Each race is a separate event which gives points towards season standing.

In [1]:
import requests
import pandas as pd
import yaml

from collections import defaultdict
from copy import deepcopy

In [2]:
from cr_racing_results import (
    RaceResults,
)

In [3]:
with open('races-config.yaml') as fp:
    race_config = yaml.safe_load(fp)

In [4]:
from pathlib import Path


official_clusters = defaultdict(tuple)

clusters_dir = Path(race_config['clusters'])
for file in list(clusters_dir.iterdir()):
    if file.name.startswith('.'):
        continue
    cluster = file.name.strip('+')
    with open(file) as fp:
        official_clusters[cluster] = official_clusters[cluster] + tuple(fp.read().split('\n'))

In [5]:
race1 = RaceResults.from_config(race_config['races'][0], official_clusters)

In [6]:
race1.save(f'data/{race1.name}')

In [13]:
race1.group['A'][race1.group['A'].name == 'Кузнецов Владимир']

,bib,name,gender,category,status,rank_abs,result,result_time,team,club
42,54,Кузнецов Владимир,male,М8,Q,43.0,02:25:13.78,8713.783,Volga Union MCA 1,Volga Union MCA 1


In [21]:
from itertools import chain
from cr_racing_results import _shorten_name
cluster_from_race = set(
    chain(*(map(_shorten_name, results.name.unique()) for results in race1.group.values()))
)

In [28]:
cluster = 'C'
current_list = official_clusters[cluster]

In [29]:
cluster_racers = set(
    map(_shorten_name, race1.group[cluster].name.unique())
)

In [32]:
race1.group['C'].loc[race1.group['C'].name.str.startswith('Кузнецов В')]

,bib,name,gender,category,status,rank_abs,result,result_time,team,club
203,748,Кузнецов Виктор,male,None,ESPLIT,NaN,—,98282.97,None,None


In [30]:
'Кузнецов В' in cluster_racers

True

In [6]:
race2 = RaceResults.from_config(race_config['races'][0], race1.clusters)
# difference

In [5]:
import os
races = []
cluster_distribution = deepcopy(official_clusters)
for race in race_config['races']:
    race = RaceResults.from_config(race, cluster_distribution)
    race.save(os.path.join('data', race.name))
    cluster_distribution = race.clusters
    races.append(race)
races.sort(key=lambda race: race.order)

In [7]:
race.group['B']

,bib,name,gender,category,status,rank_abs,result,result_time,team,club
0,114,Голубев Владимир,male,M-ABS,Q,1.0,02:48:34.18,10114.186,Клуб любителей,Клуб любителей
1,148,Усольцев Алексей,male,M-ABS,Q,2.0,02:55:00.02,10500.023,None,None
2,170,Тюриков Александр,male,M-ABS,Q,3.0,02:55:00.90,10500.902,None,None
3,182,Добродеев Сергей,male,М5 50-54,Q,4.0,02:55:01.21,10501.217,Клуб любителей,Клуб любителей
4,102,Земляной Николай,male,М2 35-39,Q,5.0,02:55:01.37,10501.370,None,None
...,...,...,...,...,...,...,...,...,...,...
78,117,Милых Станислав,male,М3 40-44,DNS,NaN,None,NaN,None,None
79,18,Наталич Павел,male,М6 55-59,DNS,NaN,None,NaN,Магадан. Кластер B,Магадан. Кластер B
80,154,Никоненко Андрей,male,М4 45-49,DNS,NaN,None,NaN,None,None
81,105,Смирнов Владислав,male,М2 35-39,DNS,NaN,None,NaN,None,None


In [6]:
cluster_standing = {}

for race in races:
    for cluster, race_points in race.get_race_points().items():
        if cluster in cluster_standing:
            cluster_standing[cluster] = pd.merge(cluster_standing[cluster], race_points, how='outer', on='name')
        else:
            cluster_standing[cluster] = race_points

In [7]:
renamed_columns = {
    "name": "Гонщик",
    "total": "Очки"
}

for cluster in cluster_standing:
    all_results = cluster_standing[cluster].set_index('name')
    all_results['total'] = all_results.sum(axis='columns')
    cluster_standing[cluster] = all_results.sort_values('total', ascending=False)


In [8]:
writer = pd.ExcelWriter('data/current_standing.xlsx', engine = 'xlsxwriter')
for cluster in cluster_standing:
    cluster_standing[cluster].reset_index().to_excel(writer, sheet_name=cluster)

    data = cluster_standing[cluster].reset_index().rename(renamed_columns, axis='columns')
    data.index += 1
    data.to_csv(f'data/cluster_{cluster}.csv')
writer.close()

In [11]:
races[1].group['B'].loc[races[1].group['B'].name.str.startswith('Есипов')]

,bib,name,gender,category,status,rank_abs,result,result_time,team,club
22,136,Есипов Евгений,male,М4 45-49,Q,23.0,02:53:18.09,10398.097,Кластер B. HBFS Plan B,Кластер B. HBFS Plan B


In [12]:
races[1].tt.loc[races[1].tt.name.str.startswith('Есипов')]

,bib,name,gender,category,status,rank_abs,result,result_time,team,club,cluster
58,179,Есипов Евгений,male,М4 45-49,Q,59.0,00:41:16.29,2476.299,HBFS,HBFS,B


In [13]:
list(filter(lambda clust: 'Есипов Е' in races[1].clusters[clust], races[1].clusters))

['B']

# Менялись ли кластеры между гонок

In [16]:
from functools import reduce
all_dfs = reduce(lambda  left,right: pd.merge(left,right, on=['name'], how='outer'), (pd.concat(dfs) for race_name, dfs in all_results.items()))

In [22]:
all_results = defaultdict(list)
for race in race_results['groups']:
    for cluster, results in race.results.items():
        set_tt_cluster(results, official_clusters, 'fixed_cluster')
        set_tt_cluster(results, original_off_clusters, 'official_cluster')
        results['real_cluster'] = cluster
        all_results[race.name].append(results.loc[:, ['name', 'real_cluster']].rename({'real_cluster': race.name}, axis=1))

In [23]:
pd.concat([
    all_dfs.loc[(all_dfs['Дубна'] != all_dfs['Верея']) & ~all_dfs['Дубна'].isna() & ~all_dfs['Верея'].isna()],
    all_dfs.loc[(all_dfs['Садовое кольцо'] != all_dfs['Верея']) & ~all_dfs['Садовое кольцо'].isna() & ~all_dfs['Верея'].isna()],
    all_dfs.loc[(all_dfs['Дубна'] != all_dfs['Верея']) & ~all_dfs['Дубна'].isna() & ~all_dfs['Верея'].isna()]
]).drop_duplicates()

,name,Садовое кольцо,Верея,Дубна
78,Сальников Роман,A,A,B
156,Карпов Антон,A,A,B
370,Дмитриев Андрей,C,C,B
712,Худиев Омар,NaN,B,A
188,Попов Сергей,C,B,B
362,Нечаев Виктор,C,B,B
548,Руез Фабьен,C,A,NaN


In [8]:
!pip install beautifulsoup4

In [9]:
from bs4 import BeautifulSoup

In [10]:
r = requests.get('https://results.zone/tula_shosse_2024/races/7357/results/148')

In [13]:
page_info = BeautifulSoup(r.text)

In [28]:
participatn_info = page_info.find_all('section', {'class': 'card-body'})

In [21]:
list(participatn_info)

[<section class="card-body" id="heat_results"><div class="row"><div class="col-8 col-xs-12 col-md-8 col-sm-6"><h3>Результат <span class="badge badge-primary">02:55:00.02</span></h3></div><div class="col-4 col-xs-12 col-md-4 col-sm-6 text-right"><h4> <span title="Темп">36.34 км/ч</span></h4></div></div><hr/><div><span class="badge badge-success">2</span> <span class="text-muted">из 69 среди всех участников</span></div><div><span class="badge badge-danger">2</span> <span class="text-muted">из 69 в абсолютном зачете среди <strong>мужчин</strong></span></div><div><span class="badge badge-warning">2</span> <span class="text-muted">из 3 в категории <strong>„M-ABS“</strong></span></div></section>,
 <section class="card-body"><dl class="row"><dt class="col-sm-3">Номер</dt><dd class="col-sm-9"><span class="badge badge-secondary badge-pill">148</span></dd><dt class="col-sm-3">Категория</dt><dd class="col-sm-9"><span class="badge badge-info">M-ABS</span></dd><dt class="col-sm-3">Год рождения</dt>

In [29]:
import re
list(card.find_all('dd', string=re.compile(r'\d{4}')) for card in participatn_info)

[[], [<dd class="col-sm-9">1999</dd>]]

In [31]:
page_info.find_all('dd', string=re.compile(r'\d{4}'))[0].text

'1999'